Goal:successfully run a hoomd.hpmc simulation of tetrahedra using the general polygon integration method 

In [65]:
import hoomd
import hoomd.hpmc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 

In [97]:
hoomd.context.initialize("--mode=cpu")

In [98]:
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=3),n=5)

notice(2): Group "all" created containing 125 particles


In [99]:
mc = hoomd.hpmc.integrate.convex_polyhedron(seed=1, d=0.1, a=0.1)

In [100]:
mc.shape_param.set('A', vertices=[(1,0,-1/np.sqrt(2)), (-1,0,1/np.sqrt(2)),(0,-1,1/np.sqrt(2)),(0,+1,1/np.sqrt(2))])

In [101]:
all = hoomd.group.all()

In [102]:
d = hoomd.dump.gsd("tetrahedra.gsd", period=(10), group=all, overwrite=True)

In [103]:
d.dump_state(obj=mc)

In [104]:
free_vol = hoomd.hpmc.compute.free_volume(mc=mc, seed=123, nsample=100)

In [105]:
hoomd.analyze.log(filename="tetrahedra.log",
                 quantities=['hpmc_free_volume'],
                            period=(100),
                            overwrite=True)

In [106]:
hoomd.run(10000)

** starting run **
Time 00:00:33 | Step 10000 / 10000 | TPS 1424.9 | ETA 00:00:00
Average TPS: 1424.65
---------
notice(2): -- HPMC stats:
notice(2): Average translate acceptance: 0.979221
notice(2): Average rotate acceptance:    0.969532
notice(2): Trial moves per second:        712215
notice(2): Overlap checks per second:     3.21336e+07
notice(2): Overlap checks per trial move: 45.1179
notice(2): Number of overlap errors:      0
-- Cell list stats:
Dimension: 0, 0, 0
** run complete **


In [18]:
data = np.genfromtxt(fname='tetrahedra.log', skip_header=True);

In [19]:
plt.figure(figsize=(4,2.2), dpi=140);
plt.plot(data[:,0], data[:,1]);

IndexError: too many indices for array

<Figure size 560x308 with 0 Axes>

In [24]:
data

array([    0.  , 13906.25])